In [1]:
#Neuroevolution, redes neurais com algoritmos genéticos, reprodução assexuada
#objetivo da IA, maximizar score relativo de N jogos.
#Cria-se uma população de M redes neurais cassineiras
#cada uma das M redes joga N jogos, 
#calcula-se o score final relativo de cada uma (que vai ser usado como função fitness),
#seleciona a com maior fitness,
#Faz-se M copias (nova população)
#muta-se as cópias
#repete seleção.

#proximos passos:
#usar uma recurrent neural network
#para ver se a rede aprende a se importar com a sequencia em que saíram as cartas
#criar funções e classes para enxugar o código
#treinar mais redes, por mais gerações e em mais jogos
#criar interface
#treinar a rede contra o cheater (que sabe de antemão as cartas que vão sair)
#colocar duas populações de redes para competirem entre si

In [2]:
#Classe Rede Neural

import numpy as np
class NeuralNetwork():
    def __init__(self):
        #Seed the random number generator, so it generates the same numbers
        #every time the program runs
        np.random.seed(7)
        
        #We model a single neuron, with 3 input connections and 1 output connection.
        #We assign random weights to a 3 x 1 matrix, with values in the range -1 to 1
        # and mean 0
        self.synaptic_weights_0 = 2 * np.random.random((4,6)) - 1
        self.synaptic_weights_1 = 2 * np.random.random((6,1)) - 1
        
        self.score_relativo = 0
        self.score_total = 0
        
    #The sigmoid function, which describes an s shaped curve
    #we pass the weighted sum of the inputs throuhg this function
    #to normalise them between 0 and 1
    
    def __sigmoid(self, x):
        return 1/(1+ np.exp(-x))
    
    #gradient of the sigmoid curve
    def __sigmoid_derivative(self, x):
        return x * (1 - x)

    
    def predict(self, inputs):
        #pass inputs through our neural network (our single neuron)
        #return self.__sigmoid(dot(inputs, self.synaptic_weights))

        #forward propagation
        l0 = inputs
        l1 = self.__sigmoid(np.dot(l0, self.synaptic_weights_0))
        l2 = self.__sigmoid(np.dot(l1, self.synaptic_weights_1))
        
        return l2

In [6]:


import random
random.seed(7)

num_redes = 1000
num_geracoes = 100



neural_network = [NeuralNetwork() for x in range(num_redes)]
super_maior_score = 0

for gen in range(num_geracoes):
    for rede in range(num_redes):
        num_jogos = 100
        a_vitorias = 0
        b_vitorias = 0
        empates = 0

        for jogo in range(num_jogos):

            baralho = [1,1,1,1, 2,2,2,2, 3,3,3,3, 4,4,4,4, 5,5,5,5, 6,6,6,6, 7,7,7,7, 8,8,8,8, 9,9,9,9,
                       10,10,10,10, 10,10,10,10, 10,10,10,10, 10,10,10,10]
            #print(len(baralho))



            a_lista = []
            b_lista = []
            a_estourou = False
            b_estourou = False


            #dar as cartas
            for i in range(2):
                a = random.choice(baralho)
                a_lista.append(a)
                try:
                    baralho.remove(a)
                except:
                    pass

                b = random.choice(baralho)
                b_lista.append(b)
                try:
                    baralho.remove(b)
                except:
                    pass

            #print("Jogador a: cartas", a_lista, "pontos", sum(a_lista))
            #print("Jogador b: cartas", b_lista, "pontos", sum(b_lista))

            #vez de a
            while(a_estourou == False):

                a_continuar = bool(random.getrandbits(1))
                #print("")
                #print("Cartas: ", a_lista, "Total: ", sum(a_lista))
                #a_continuar = bool(int(input("Parar 0 ou Continuar 1? ")))
                #print(a_continuar)


                if (a_continuar == True):
                    if (len(baralho) <= 0):
                        break

                    a = random.choice(baralho)
                    a_lista.append(a)

                    try:
                        baralho.remove(a)
                    except:
                        pass

                    if sum(a_lista) > 21:
                        a_estourou = True
                        #print("Estourou!")
                else:
                    break

            #vez de b        
            while(b_estourou == False):


                #continuar = recurrentneuralnet(lista de cartas, pontos)
                inputs = np.array([(sum(b_lista))/30, (num_jogos - jogo)/num_jogos, (b_vitorias - a_vitorias)/num_jogos, 1])
                b_rede_out = neural_network[rede].predict(inputs)

                if b_rede_out < 0.5:
                    b_continuar = True
                else:
                    b_continuar = False

                #print(b_rede_out, b_continuar)
                #b_continuar = bool(random.getrandbits(1))
                if (b_continuar == True):
                    if (len(baralho) <= 0):
                        break


                    b = random.choice(baralho)
                    b_lista.append(b)
                    try:
                        baralho.remove(b)
                    except:
                        pass

                    if sum(b_lista) > 21:
                        b_estourou = True
                else:
                    break



            a_soma = sum(a_lista)
            b_soma = sum(b_lista)

            #print("")
            #print("Jogo", jogo)

            #print("Jogador a: cartas", a_lista, "pontos", a_soma)
            #print("Jogador b: cartas", b_lista, "pontos", b_soma)


            if ( (a_soma > b_soma) & (a_soma <= 21) ) | ( (b_estourou == True) & (a_estourou == False) ):
                #print("Jogador a ganhou!")
                a_vitorias += 1
            elif ( (b_soma > a_soma ) & (b_soma <= 21) ) | ( (a_estourou == True) & (b_estourou == False) ):
                #print("Jogador b ganhou!") 
                b_vitorias += 1
            else:
                #print("Empate!")
                empates += 1

        #print("")
        #print("Campeonato", rede, "Resultado Final.", num_jogos, "jogos", gen, "geração")
        #print("Jogador a:", a_vitorias, "vitórias.")
        #print("Jogador b:", b_vitorias, "vitórias.")
        #print("Empates", empates)

        #if(a_vitorias > b_vitorias):
            #print("Jogador a ganhou! ", "score relativo: ", a_vitorias - b_vitorias)
        #elif(b_vitorias > a_vitorias):
            #print("Jogador b ganhou! ", "score relativo: ", b_vitorias - a_vitorias)
        #else:
            #print("Empataram!", "score_relativo:", b_vitorias - a_vitorias)
        
        neural_network[rede].score_relativo = b_vitorias - a_vitorias
        neural_network[rede].score_total = b_vitorias
    
    print("Geração", gen)
    maior_score = 0
    melhor_rede = 0
    total_score = 0
    for i in range(num_redes):
        if neural_network[i].score_relativo > maior_score:
            maior_score = neural_network[i].score_relativo
            total_score = neural_network[i].score_total
            melhor_rede = i
        #print("rede",i, ":", neural_network[i].score_relativo)
    print("Score Total:", total_score, "de", num_jogos, "jogos")
    #print("Melhor rede:", melhor_rede)
    
    if total_score > super_maior_score:
        super_maior_score = total_score
        syn1 = np.copy(neural_network[melhor_rede].synaptic_weights_1)
        syn0 = np.copy(neural_network[melhor_rede].synaptic_weights_0)
    
    print("Melhor Score de todos:", super_maior_score)
    
    for i in range(num_redes):
        if i != melhor_rede:
            #copia
            #nao faça esse tipo de cópia! não é cópia! é só redefinição de ponteiro!
            #neural_network[i].synaptic_weights_0 =  neural_network[melhor_rede].synaptic_weights_0
            #neural_network[i].synaptic_weights_1 =  neural_network[melhor_rede].synaptic_weights_1
            ###
            #cópia
            neural_network[i].synaptic_weights_0 = np.copy(neural_network[melhor_rede].synaptic_weights_0)
            neural_network[i].synaptic_weights_1 = np.copy(neural_network[melhor_rede].synaptic_weights_1)
            
            #mutacao
            alpha_x = random.choice(range(np.size(neural_network[i].synaptic_weights_0,0)))
            alpha_y = random.choice(range(np.size(neural_network[i].synaptic_weights_0,1)))
            neural_network[i].synaptic_weights_0[alpha_x][alpha_y] =  2 * np.random.rand() - 1 # #neural_network[i].synaptic_weights_0[alpha_x][alpha_y] * (np.random.rand() * 2)
            
            alpha_x = random.choice(range(np.size(neural_network[i].synaptic_weights_1,0)))
            alpha_y = random.choice(range(np.size(neural_network[i].synaptic_weights_1,1)))
            neural_network[i].synaptic_weights_1[alpha_x][alpha_y] = 2 * np.random.rand() - 1 # neural_network[i].synaptic_weights_1[alpha_x][alpha_y] * (np.random.rand() * 2) # #
       
        #print("Rede", i)
        #print(neural_network[i].synaptic_weights_1)
        
    #print("Melhor rede")
    #print(neural_network[melhor_rede].synaptic_weights_1)    

print("")
print("Melhor rede syn 1")
print(syn1)
print("Melhor rede syn 0")
print(syn0)

Geração 0
Score Total: 0 de 100 jogos
Melhor Score de todos: 0
Geração 1
Score Total: 77 de 100 jogos
Melhor Score de todos: 77
Geração 2
Score Total: 81 de 100 jogos
Melhor Score de todos: 81
Geração 3
Score Total: 77 de 100 jogos
Melhor Score de todos: 81
Geração 4
Score Total: 76 de 100 jogos
Melhor Score de todos: 81
Geração 5
Score Total: 74 de 100 jogos
Melhor Score de todos: 81
Geração 6
Score Total: 80 de 100 jogos
Melhor Score de todos: 81
Geração 7
Score Total: 78 de 100 jogos
Melhor Score de todos: 81
Geração 8
Score Total: 78 de 100 jogos
Melhor Score de todos: 81
Geração 9
Score Total: 76 de 100 jogos
Melhor Score de todos: 81
Geração 10
Score Total: 77 de 100 jogos
Melhor Score de todos: 81
Geração 11
Score Total: 77 de 100 jogos
Melhor Score de todos: 81
Geração 12
Score Total: 73 de 100 jogos
Melhor Score de todos: 81
Geração 13
Score Total: 83 de 100 jogos
Melhor Score de todos: 83
Geração 14
Score Total: 78 de 100 jogos
Melhor Score de todos: 83
Geração 15
Score Total

In [14]:
print(2 * np.random.rand() - 1)

0.046825161347531674


In [20]:
len(baralho)

48


Jogo 0
Jogador a: cartas [6, 7] pontos 13
Jogador b: cartas [3, 10] pontos 13
Empate!

Jogo 1
Jogador a: cartas [9, 7] pontos 16
Jogador b: cartas [2, 10, 9] pontos 21
Jogador b ganhou!

Jogo 2
Jogador a: cartas [1, 8] pontos 9
Jogador b: cartas [2, 8] pontos 10
Jogador b ganhou!

Jogo 3
Jogador a: cartas [2, 8, 10] pontos 20
Jogador b: cartas [10, 1, 5, 10] pontos 26
Jogador a ganhou!

Jogo 4
Jogador a: cartas [10, 10] pontos 20
Jogador b: cartas [1, 10] pontos 11
Jogador a ganhou!

Jogo 5
Jogador a: cartas [4, 10] pontos 14
Jogador b: cartas [1, 3] pontos 4
Jogador a ganhou!

Resultado Final
Jogador a: 3 vitórias!
Jogador b: 2 vitórias!
Jogador a ganhou.  score relativo:  1


In [3]:
bool

False

Melhor rede syn 1
[[-0.5046907 ]
 [-0.12193648]
 [ 0.13441028]
 [-0.00442857]
 [ 0.84432307]
 [-0.10388107]]
Melhor rede syn 0
[[-0.33578113  0.55983758 -0.3643266  -0.5507968   0.97797735  0.80181873]
 [ 0.00224093 -0.12310751 -0.93036136 -0.8768388  -0.26311909 -0.57066169]
 [-0.09515334 -0.95599412 -0.99222458 -0.44341849 -0.43486865 -0.10980755]
 [-0.09121938 -0.63684532  0.15594223 -0.60178605 -0.98598994 -0.40111507]]